In [141]:
import requests
import nyc_geoclient
from nyc_geoclient import Geoclient
import mplleaflet as mpll

import numpy as np
import pandas as pd
#import geopandas as gp
import folium as fl
import seaborn as sns
import pylab as plt
from IPython.display import HTML
% pylab inline

import geopandas as gpd

from geopandas import GeoSeries, GeoDataFrame
from shapely.geometry import Point

Populating the interactive namespace from numpy and matplotlib


In [69]:
path = "/projects/18_predicting_igc/shared/data/"

# I registered this ID and key to the GeoCoder API for our project, so use it if you wish

app_ID = "ebc3a911"
app_key = "4bf9b57657e7fedb10b6998182f588e6"
g = Geoclient(app_ID, app_key)

In [132]:
df = pd.read_csv(path + 'inspections_master_v1_1.csv')
df.head()

,Unnamed: 0,priority_code,community_board,address,ZIP,borough,BIN,ID,complaint_category,complaint_category_description,...,referral_description,referral_agency_code,complaint_source_description.1,complaint_reason,disp_code,disp_description,inspector_comments,ECB_violation_number,outcomes,insp_date
0,0,B,102,110 CHRISTOPHER STREET,10014.0,Manhattan,1010038,1272891,66,PLUMBING WORK - ILLEGAL/NO PERMIT(ALSO SPRINKL...,...,NaN,NaN,NaN,PLUMBING WORK IS ONGOING WITHOUT FILING FOR PE...,A8,ECB VIOLATION SERVED,AS PER INSPECTOR,34806614Z,2,2010-01-04
1,2,C,110,2492 7 AVENUE,10030.0,Manhattan,1060459,1272972,94,PLUMBING-DEFECTIVE/LEAKING/NOT MAINTAINED,...,NaN,NaN,NaN,CALLER STATES THE PIPES ARE FROZEN IN THE BEAU...,A8,ECB VIOLATION SERVED,AS PER INSPECTOR,34806710Z,2,2010-01-06
2,3,B,101,116 JOHN STREET,10038.0,Manhattan,1001126,1273017,66,PLUMBING WORK - ILLEGAL/NO PERMIT(ALSO SPRINKL...,...,NaN,NaN,NaN,CALLER IS REPORTING THAT OWNER'S IN FRONT OF A...,D2,COMPLAINT ASSIGNED TO PLUMBING UNIT,REFERRED TO DOB PLUMBING UNIT,NaN,1,2010-04-06
3,5,C,110,2445 FREDERICK DOUGLAS BOULEVARD,10027.0,Manhattan,1059412,1273032,94,PLUMBING-DEFECTIVE/LEAKING/NOT MAINTAINED,...,NaN,NaN,NaN,NO HOT WATER AT THE SALON. FOR OVER TWO WEEKS....,I2,NO VIOLATION WARRANTED FOR COMPLAINT AT TIME O...,"NO VIOLATION WARRANTED AT TIME OF INSPECTION, ...",NaN,1,2010-04-02
4,6,A,103,420 EAST 10 STREET,10009.0,Manhattan,1004547,1273060,65,GAS HOOK-UP/PIPING - ILLEGAL OR DEFECTIVE,...,NaN,NaN,NaN,WORKING ON GAS PIPE AGAINST STOP WORK,A3,FULL STOP WORK ORDER SERVED ...,"ECB VIOLATION ISSUED, WORK W/O A PERMIT",NaN,2,2010-01-05


In [133]:
df_EV = df[df.community_board==103]  # the East Village
df_EV.shape

(745, 24)

In [134]:
df_EV.reset_index(drop=True,inplace=True)
df_EV['lat']=""
df_EV['lon']=""
df_EV.head()

/nfshome/nj935/.conda/envs/remote_sensing/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nfshome/nj935/.conda/envs/remote_sensing/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Unnamed: 0,priority_code,community_board,address,ZIP,borough,BIN,ID,complaint_category,complaint_category_description,...,complaint_source_description.1,complaint_reason,disp_code,disp_description,inspector_comments,ECB_violation_number,outcomes,insp_date,lat,lon
0,6,A,103,420 EAST 10 STREET,10009.0,Manhattan,1004547,1273060,65,GAS HOOK-UP/PIPING - ILLEGAL OR DEFECTIVE,...,NaN,WORKING ON GAS PIPE AGAINST STOP WORK,A3,FULL STOP WORK ORDER SERVED ...,"ECB VIOLATION ISSUED, WORK W/O A PERMIT",NaN,2,2010-01-05,,
1,28,C,103,206 AVENUE A,10009.0,Manhattan,1005202,1274176,94,PLUMBING-DEFECTIVE/LEAKING/NOT MAINTAINED,...,NaN,CUSTOMER STATES THAT THE LANDLORD DAMAGED HIS ...,C1,INSPECTOR UNABLE TO GAIN ACCESS - 1ST ATTEMPT,"COULD NOT SEE ANY DAMAGE, LEFT LS-4 FOR OWNER ...",NaN,1,2010-02-04,,
2,38,C,103,438 EAST 9 STREET,10009.0,Manhattan,1005880,1274789,94,PLUMBING-DEFECTIVE/LEAKING/NOT MAINTAINED,...,NaN,LEAK IN RESTAURANT COMING FROM UPSTAIRS,C1,INSPECTOR UNABLE TO GAIN ACCESS - 1ST ATTEMPT,LS-4 POSTED,NaN,0,2010-02-09,,
3,66,C,103,60 HENRY STREET,10002.0,Manhattan,1077812,1275425,94,PLUMBING-DEFECTIVE/LEAKING/NOT MAINTAINED,...,NaN,CALLER IS REPORTING A MAINTENANCE PROBLEM AT T...,I2,NO VIOLATION WARRANTED FOR COMPLAINT AT TIME O...,NO VIOLATION WARRANTED AT TIME OF INSPECTION,NaN,1,2010-02-09,,
4,79,B,103,70 FORSYTH STREET,10002.0,Manhattan,1003949,1275941,66,PLUMBING WORK - ILLEGAL/NO PERMIT(ALSO SPRINKL...,...,NaN,BEHIND THE IRON GATE OF THE RESTAURANT THEY A...,B2,ECB VIOLATION(S) PREPARED & ATTEMPT TO SERVE W...,AS PER INSPECTOR,34745069X,2,2010-02-22,,


In [136]:
for i, BIN in enumerate(df_EV.BIN[:350]):
    df_EV.lat[df_EV.BIN == BIN] = g.bin(df_EV.BIN[i])['latitudeInternalLabel']
    df_EV.lon[df_EV.BIN == BIN] = g.bin(df_EV.BIN[i])['longitudeInternalLabel']

/nfshome/nj935/.conda/envs/remote_sensing/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nfshome/nj935/.conda/envs/remote_sensing/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [109]:
df_EV.columns

Index([u'Unnamed: 0', u'priority_code', u'community_board', u'address', u'ZIP',
       u'borough', u'BIN', u'ID', u'complaint_category',
       u'complaint_category_description', u'complaint_source',
       u'complaint_source_description', u'date_received', u'd_date',
       u'referral_description', u'referral_agency_code',
       u'complaint_source_description.1', u'complaint_reason', u'disp_code',
       u'disp_description', u'inspector_comments', u'ECB_violation_number',
       u'outcomes', u'insp_date', u'lat', u'lon'],
      dtype='object')

In [137]:
df_EV.lat = pd.to_numeric(df_EV.lat)
df_EV.lon = pd.to_numeric(df_EV.lon)

/nfshome/nj935/.conda/envs/remote_sensing/lib/python2.7/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [142]:
s = gpd.GeoSeries([Point(x,y) for x, y in zip(df_EV['lon'], df_EV['lat'])])
geo_df = gpd.GeoDataFrame(df_EV,geometry = s)
geo_df.crs = {'init': 'epsg:4326'} 

In [147]:
fig, ax = plt.subplots(1, figsize=(12,12))
geo_df[geo_df.outcomes==0].plot(c='y',ax=ax)
geo_df[geo_df.outcomes==1].plot(c='g',ax=ax)
geo_df[geo_df.outcomes==2].plot(c='r',ax=ax)
mpll.display(fig=fig, crs = geo_df.crs, tiles='cartodb_positron')